# Нейронные сети

## Задача
Реализовать классификационную или регрессионную модель нейронной сеть, используя только средства numpy (и matplotlib для визуализации):
1. Модель нейронной сети должна состоять из двух скрытых (внутренних) слоев ($k_1$ и $k_2$ - число нейронов скрытых слоев соответсвенно). 
2. Число нейронов в скрытых слоях выберите самостоятельно. Желательно реализовать выбор количества нейронов в скрытых слоях.
3. Активационные функции в скрытых слоях выберите самостоятельно ('linear', 'sigmoid', 'tanh', 'relu'). 
4. Самостоятельно выбрать датасет для построения модели. Можно воспользоваться следующими данными:
    - breast cancer wisconsin dataset (classification) sklearn.datasets.load_breast_cancer
    - boston house-prices dataset (regression) sklearn.datasets.load_boston
    - diabetes dataset (regression) sklearn.datasets.load_diabetes
    - digits dataset (classification) sklearn.datasets.load_digits
    Отвести 70% данных под тренировочную выборку, 20% под валидационную, 10% под тестовую.
5. Реализовать прямой (forward) и обратный (backward) проходы по нейронной сети.
6. Выбрать и реализовать оценки качества реализуемой модели (для классификации: accuracy, precision, recall, Fscore; для регрессии: mean absolute error (MAE))
7. Обучить нейронную сеть на тренировочном наборе данных, на каждой эпохе обучения валидировать данные. Другими словами, необходимо выводить для каждой эпохи значения функции потерь и метрик точности для тренировочного и валидационного набора данных. Метрики необходимо строить на одном графике для оценки качества построенной модели.
8. После тренировки модели необходимо протестировать модель на оставшемся наборе данных и сравнить все метрики для каждого набора данных.
9. Сделать выводы

Подсказки:
- Обязательно сделайте нормализацию данных.
- Веса внутренних слоев сгенерируйте случайным образов в диапазоне от $[-1, 1]$.


In [93]:
#X = data.data[:354]
#Y = data.target[:354]
#validX = data.data[354:455]
#validY = data.target[354:455]
#testX = data.data[455:]
#testY = data.target[455:]

In [20]:
#Y_train_class = MinMaxScaler().fit_transform(Y_train_class.reshape(-1,1))
#Y_train_regres = Y_train_regres.reshape(-1,1)

In [1]:
from sklearn.datasets import load_boston,load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
seed = 1
data = load_boston()
def tahn(x, deriv = False):
    if deriv == True:
        return 1 - tahn(x)**2
    return 2 / (1 + np.exp(-2*x)) - 1

def sigmoid(x, deriv = False):
    if deriv == True:
        return sigmoid(x)*(1-sigmoid(x))
    return 1 / (1 + np.exp(-x))
def relu(x, deriv = False):
    arr = x > 0
    if deriv == True:
        return arr.astype(float)
    return arr.astype(float) * x
def softmax(x, deriv = False):
    if deriv == True:
        return np.array([it * (1 - it) for it in x])
    return np.array([np.exp(it) / np.sum(np.exp(it)) for it in x])

In [3]:
def categories(Y):
    enc_y = []
    for a in Y:
        if a > 0.5:
            enc_y.append([0,1])       
        else:
            enc_y.append([1,0])
    return np.array(enc_y)
def cross_entropy(y,q):
    return -(y*np.log2(q)+(1-y)*np.log2(1-q))

In [4]:
X = data.data
Y = data.target
activation = sigmoid
X = MinMaxScaler().fit_transform(X)
Y = MinMaxScaler().fit_transform(Y.reshape(-1,1))
Y_class = categories(Y)

In [5]:
test_size = 0.25
X_train_class, X_test_class, Y_train_class, Y_test_class     = train_test_split(X, Y, test_size=test_size, random_state=seed)
X_train_regres, X_test_regres, Y_train_regres, Y_test_regres = train_test_split(X, Y_class, test_size=test_size, random_state=seed)

In [17]:
class NeuralNetwork:
    def __init__(self, x, y_class, y_regres):
        # Входные данные #
        np.random.seed(1)
        self.input         = x
        self.y_class       = y_class        
        self.y_regres      = y_regres
        self.output_class  = np.zeros(len(y_class))
        self.output_regres = np.zeros(len(y_regres))        
        self.lr = 0.001
        self.weights_1 = 2*np.random.random((13,8)) - 1
        self.weights_2_class = 2*np.random.random((8,2)) - 1
        self.weights_2_regres = 2*np.random.random((8,4)) - 1
        self.weights_out_class = 2*np.random.random((2,1)) - 1      
        self.weights_out_regres = 2*np.random.random((4,2)) - 1      
        # Прямой проход #
    def FeedForward(self, x):            
        self.out1 = activation(np.dot(x,self.weights_1))
        self.out2_class = activation(np.dot(self.out1,self.weights_2_class))  
        self.out2_regres = activation(np.dot(self.out1, self.weights_2_regres))
        self.out_class = np.dot(self.out2_class,self.weights_out_class)#activation(
        self.out_regres = softmax(np.dot(self.out2_regres, self.weights_out_regres))
        return self.out_regres,self.out_class
        
        # Обратный проход #
    def BackProp(self, x, y_class,y_regres):     
        
        out_error_3 = y_class-self.out_class
        d_weights_3 = out_error_3 * activation(self.out_class, deriv=True)           
        out_error_2 = d_weights_3.dot(self.weights_out_class.T)
        d_weights_2 = out_error_2 * activation(self.out2_class, deriv=True)
        out_error_1 = d_weights_2.dot(self.weights_2_class.T)
        d_weights_1 = out_error_1 * activation(self.out1, deriv=True)
        
        out_error_3_reg = y_regres - self.out_regres
        d_weights_3_reg  = out_error_3_reg * softmax(self.out_regres,True)#
        out_error_2_reg = d_weights_3_reg.dot(self.weights_out_regres.T)
        d_weights_2_reg = out_error_2_reg * activation(self.out2_regres, deriv=True)
        out_error_1_reg = d_weights_2_reg.dot(self.weights_2_regres.T)
        d_weights_1_reg = out_error_1_reg * activation(self.out1, deriv=True)

        self.weights_1 += self.lr * self.input.T.dot(d_weights_1)
        self.weights_1 += self.lr * self.input.T.dot(d_weights_1_reg)
        self.weights_2_class += self.lr * np.dot(self.out1.T, d_weights_2)
        self.weights_2_regres += self.lr * np.dot(self.out1.T, d_weights_2_reg)
        self.weights_out_class += self.lr * np.dot(self.out2_class.T, d_weights_3)
        self.weights_out_regres += self.lr * np.dot(self.out2_regres.T, d_weights_3_reg)
        
        # Тренировка #
    def train(self, x, y_class, y_regres):
        self.output_class, self.output_regres = self.FeedForward(x)#        
        self.BackProp(x, y_class, y_regres)
        
    def Predict(self, x, y_class, y_regres):
        print(x)
        out1 = activation(np.dot(x,self.weights_1))
        out2_class = activation(np.dot(out1,self.weights_2_class))  
        out2_regres = activation(np.dot(out1, self.weights_2_regres))
        out_class = np.dot(out2_class,self.weights_out_class)#activation(
        out_regres = softmax(np.dot(out2_regres, self.weights_out_regres))
        return out_class,out_regres

In [27]:
nn = NeuralNetwork(X_train_class,Y_train_class,Y_train_regres)
epoch = 6500
for i in range(epoch):
    nn.train(X_train_class,Y_train_class,Y_train_regres)
    if (i% round(epoch/10)) == 0:     
        print ("Class Loss: \n" + str(np.mean(cross_entropy(Y_train_regres,nn.out_regres))))  
        print ("Regres Loss: \n" + str(np.mean(np.square(Y_train_class - nn.out_class)))) # mean sum squared loss
        print("*******************************")

Class Loss: 
1.3529271905023077
Regres Loss: 
0.6279200728286882
*******************************
Class Loss: 
0.693521921105171
Regres Loss: 
0.043320311207145204
*******************************
Class Loss: 
0.6352042365318371
Regres Loss: 
0.04040717647956292
*******************************
Class Loss: 
0.5663899859828474
Regres Loss: 
0.03343172067318924
*******************************
Class Loss: 
0.5176791174257699
Regres Loss: 
0.025529883489256784
*******************************
Class Loss: 
0.47566592628980653
Regres Loss: 
0.021086734212299396
*******************************
Class Loss: 
0.40922450755481177
Regres Loss: 
0.01760239413956523
*******************************
Class Loss: 
0.3607912620319978
Regres Loss: 
0.015217140318240925
*******************************
Class Loss: 
0.33730966331585216
Regres Loss: 
0.0140657812144276
*******************************
Class Loss: 
0.32511329793672455
Regres Loss: 
0.013443731900878995
*******************************


In [28]:
print("Actual Class Output: \n" + str(Y_train_regres[:10]))
print("Predicted Class Output: \n" + str(nn.out_regres[:10].round()))
print ("Actual Regres Output: \n" + str(Y_train_class[:10]))
print ("Predicted Regres Output: \n" + str(nn.out_class[:10]))

Actual Class Output: 
[[1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]
Predicted Class Output: 
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]
Actual Regres Output: 
[[0.34666667]
 [0.40222222]
 [0.51111111]
 [0.33333333]
 [0.40222222]
 [0.44444444]
 [0.10444444]
 [0.42      ]
 [0.69111111]
 [0.18666667]]
Predicted Regres Output: 
[[0.36946975]
 [0.38421021]
 [0.51263743]
 [0.36245198]
 [0.41672496]
 [0.43956037]
 [0.21285166]
 [0.45760779]
 [0.6470444 ]
 [0.23969459]]


In [97]:
y_pred_class, y_pred_regres = nn.Predict(X_test_class, Y_test_class, Y_test_regres)

[[4.83309632e-04 3.30000000e-01 6.30498534e-02 ... 6.17021277e-01
  1.00000000e+00 1.60044150e-01]
 [2.14791792e-04 5.50000000e-01 1.21700880e-01 ... 5.31914894e-01
  1.00000000e+00 1.50386313e-01]
 [2.50590425e-03 0.00000000e+00 2.36436950e-01 ... 5.63829787e-01
  9.89510313e-01 4.71026490e-01]
 ...
 [4.11937163e-04 5.25000000e-01 1.78152493e-01 ... 4.25531915e-01
  9.36507136e-01 2.14679912e-01]
 [2.61661587e-03 0.00000000e+00 3.38343109e-01 ... 7.02127660e-01
  1.00000000e+00 3.08774834e-01]
 [8.57930796e-04 0.00000000e+00 4.93401760e-01 ... 3.61702128e-01
  1.00000000e+00 1.89017660e-01]]


In [98]:
print ("Class Loss: \n" + str(np.mean(cross_entropy(Y_train_regres,nn.out_regres))))  
print ("Regres Loss: \n" + str(np.mean(np.square(Y_train_class - nn.out_class))))

Class Loss: 
0.3133735727167919
Regres Loss: 
0.012752710700940514


In [95]:
epoch = 1500
lr = 0.1
np.random.seed(1)
weights_1 = 2*np.random.random((13,25)) - 1
weights_2 = 2*np.random.random((25,20)) - 1
weights_3 = 2*np.random.random((20,1)) - 1

error = []
activation = sigmoid
for i in range(epoch):
    inp = X
    out1 = activation(np.dot(inp,weights_1))
    out2 = activation(np.dot(out1,weights_2))  
    out3 = np.dot(out2,weights_3)
    
    out_error = (out3 - Y)**2
    error.append(np.mean(out_error))
    
    out_error_3 = Y-out3    
    d_weights_3 = out_error_3 * activation(out3, deriv=True)   
    out_error_2 = d_weights_3.dot(weights_3.T)
    d_weights_2 = out_error_2 * activation(out2, deriv=True)
    out_error_1 = d_weights_2.dot(weights_2.T)
    d_weights_1 = out_error_1 * activation(out1, deriv=True)
    
    
    weights_1 += lr*inp.T.dot(d_weights_1)
    weights_2 += lr * np.dot(out1.T, d_weights_2)
    weights_3 += lr * np.dot(out2.T, d_weights_3)
    if (i% round(epoch/10)) == 0:
        print("Ответы")
        print(Y[:10])
        print("Выход после тренировки")
        print(out3[:10])
        print ("Class Loss: \n" + str(np.mean(np.square(Y - out3))))
#plt.plot(error)
#print("Ответы")
#print(y[:10])
#print("Выход после тренировки")
#print(out3[:10])

Ответы
[[0.31937173]
 [0.2565445 ]
 [0.59947644]
 [0.56544503]
 [0.63874346]
 [0.44240838]
 [0.29057592]
 [0.40052356]
 [0.12303665]
 [0.18586387]]
Выход после тренировки
[[-1.92135747]
 [-1.81462606]
 [-1.83578216]
 [-1.85073576]
 [-1.85609089]
 [-1.82688892]
 [-1.930663  ]
 [-1.91302331]
 [-1.91105521]
 [-1.91845563]]
Class Loss: 
4.94681579943799
Ответы
[[0.31937173]
 [0.2565445 ]
 [0.59947644]
 [0.56544503]
 [0.63874346]
 [0.44240838]
 [0.29057592]
 [0.40052356]
 [0.12303665]
 [0.18586387]]
Выход после тренировки
[[38.75080016]
 [38.99422554]
 [38.74209474]
 [38.45860368]
 [38.48621388]
 [38.64074524]
 [39.8505597 ]
 [39.97472207]
 [40.21985722]
 [40.00071926]]
Class Loss: 
1536.3470484648856
Ответы
[[0.31937173]
 [0.2565445 ]
 [0.59947644]
 [0.56544503]
 [0.63874346]
 [0.44240838]
 [0.29057592]
 [0.40052356]
 [0.12303665]
 [0.18586387]]
Выход после тренировки
[[38.75080016]
 [38.99422554]
 [38.74209474]
 [38.45860368]
 [38.48621388]
 [38.64074524]
 [39.8505597 ]
 [39.97472207]
 [4